In [46]:
from __future__ import division
import itertools
from random import randint, seed
#from sage.all import gcd, inverse_mod

In [47]:
# https://en.wikibooks.org/wiki/Algorithm_Implementation/Mathematics/Extended_Euclidean_algorithm
def xgcd(b, n):
    x0, x1, y0, y1 = 1, 0, 0, 1
    while n != 0:
        q, b, n = b // n, n, b % n
        x0, x1 = x1, x0 - q * x1
        y0, y1 = y1, y0 - q * y1
    return  b, x0, y0

def gcd(a, b):
    return xgcd(a, b)[0]

def inverse_mod(r, q):
    assert gcd(r, q) == 1
    res = xgcd(r, q)[1]
    if res < 0:
        res += q
    assert res > 0 and res < q
    return res

In [90]:
def knapsack_brute_force(knapsack, x, solution):
    
    if x == 0:
        return [True, solution]
    
    if len(knapsack) == 0:
        if x == 0:
            return [True, solution]
        else:
            print 'c'
            return [False, []]
    
    if len(knapsack) == 1:
        if knapsack[0] == x:
            solution.append(x)
            return [True, solution]
        else:
            return [False, ]
    
    
    curr_elem = knapsack[0]
    
    if curr_elem > x:
        return knapsack_brute_force(knapsack[1:], x, solution)
    
    else:
        # try with taking the first element
        result = knapsack_brute_force(knapsack[1:], x - curr_elem, solution)
        if result[0]:
            solution.append(curr_elem)
            return [True, solution]
        else:
            # try with not taking the first element
            return knapsack_brute_force(knapsack[1:], x, solution)
        
def knapsack_brute_force_bits(knapsack, x):
    sol = knapsack_brute_force(knapsack, x, [])[1]
    res = []
    # dont ask my why in inverse order :D
    for i in range(1, len(knapsack) + 1):
        if knapsack[-i] in sol:
            res.append(1)
        else:
            res.append(0)
    return res

In [91]:
def new_mochila(N=10, jump=10, sd=0):
    seed(sd)
    diff = [randint(1,jump) for k in range(N)]
    mochila = [diff[0]]
    for i in range(1,len(diff)):
        new_val = sum(mochila) + diff[i]
        mochila.append(new_val)
    return mochila

def split2len(s, n=16):
    def _f(s, n):
        while s:
            yield s[:n]
            s = s[n:]
    return list(_f(s, n))

def text_to_bytes(text, n=0):
    largo = len(text)
    if n > 0:
        text = text + ' '*(n - (largo % n))
    text = text.decode('utf8')
    text_e = ''.join(['{0:016b}'.format(ord(c)) for c in text])
    return text_e

def bytes_to_text(binary):
    return ''.join([unichr(int(ch,2)) for ch in split2len(binary)])

def select_q(mochila):
    suma = sum(mochila)
    return randint(suma+1,suma*10)

def select_r(q):
    try_r = randint(2,q-1)
    while gcd(try_r,q) != 1:
        try_r = randint(2,q)
    return try_r, inverse_mod(try_r,q)

def mochila_camuflada(mochila, q, r):
    return [(r*x)% q for x in mochila]

def codificar(mochila_camuflada, text):
    largo_mochila = len(mochila_camuflada)
    assert largo_mochila % 16 == 0
    
    texto_codificado = text_to_bytes(text,largo_mochila // 16)
    a_codifcar = split2len(texto_codificado, largo_mochila)
    return [sum([b for (a,b) in zip(k,mochila_camuflada) if a=='1']) for k in a_codifcar], texto_codificado


def unpack(n,l):
    i = len(l) -1
    for i in range(len(l)-1,-1,-1):
        if n>0:
            if l[i] <= n:
                n -= l[i]
                res = 1
            else:
                res = 0
            i-=1
        else:
            res = 0
        yield res

def decodificar(chunk, mochila_o, q, i_r):
    message = (chunk * i_r) % q
    int_message = list(unpack(message,mochila_o))[::-1]
    return bytes_to_text(''.join(map(str,int_message)))

def decodificar_brute_force(chunk, mochi_c):
    int_message = list(knapsack_brute_force_bits(mochi_c, chunk))[::-1]
    return bytes_to_text(''.join(map(str,int_message)))
    


In [92]:
mochi = new_mochila(N=16, jump=5)
print mochi
q = select_q(mochi)
r, i_r = select_r(q)
mochi_c = mochila_camuflada(mochi, q, r)
print mochi_c

texto = 'Nó mé gústÁn lás tÍldÉs'
codifcado, x= codificar(mochi_c, texto)
print codifcado, x

[5, 9, 17, 33, 67, 134, 269, 536, 1073, 2146, 4294, 8586, 17171, 34344, 68688, 137374]
[2307014, 2133194, 1785554, 1090274, 2137093, 1749897, 932050, 1951010, 1334276L, 144263L, 201616L, 490142L, 1023739L, 1960568L, 1396847L, 356315L]
[4525417L, 3923459L, 201616L, 3686501L, 3060209L, 201616L, 4059609L, 4590883L, 2589183L, 2796589L, 1834854L, 4727033L, 201616L, 3330186L, 2036470L, 2589183L, 201616L, 2796589L, 4819161L, 3330186L, 2306447L, 2858593L, 2589183L, 201616L] 000000000100111000000000111100110000000000100000000000000110110100000000111010010000000000100000000000000110011100000000111110100000000001110011000000000111010000000000110000010000000001101110000000000010000000000000011011000000000011100001000000000111001100000000001000000000000001110100000000001100110100000000011011000000000001100100000000001100100100000000011100110000000000100000


In [93]:
s=''
for c in codifcado:
    s+=decodificar(c,mochi, q, i_r)
print s

Nó mé gústÁn lás tÍldÉs 


In [94]:
s=''
for c in codifcado:
    s+=decodificar_brute_force(c,mochi_c)
print s

Nó mé gústÁn lás tÍldÉs 
